# 深層学習を用いた言語モデル

*Deep Learning*

深層学習を用いて言語モデルを作成する。  
ニューラルネットワークを用いて、ある単語から次の単語を予測するモデルを作成し、深層学習を活用した言語モデル実装の基礎を学ぶ。

In [1]:
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import random
from typing import List

import tensorflow as tf
import tensorflow_datasets as tfds
import MeCab
import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchtext.vocab import vocab, build_vocab_from_iterator
from dlprog import train_progress

In [2]:
prog = train_progress()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')


---

## データセット

wiki40b

In [3]:
with tf.device('/cpu:0'):
    ds = tfds.load('wiki40b/ja', split='test')
ds = list(ds.as_numpy_iterator())

In [4]:
data = []
for sample in ds:
    text = sample['text'].decode()
    sections = text.split('_START_SECTION_')
    for section in sections[1:]:
        sentence = section.split('_START_PARAGRAPH_')[1]
        sentence = sentence.replace('_NEWLINE_', '')
        sentence = sentence.replace('\n', '')
        data.append(sentence)

print('num of data:', len(data))
data[:5] # examples

num of data: 89698


['「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。',
 'ライブドア社員であった初代代表取締役社長の山名真由によって企業内起業の形で創業。2005年に株式会社ライブドアから分割されて設立。かつてはライブドアホールディングス（現・LDH）の子会社であったが、ノンコア事業の整理にともない、株式会社ゲオ（現：株式会社ゲオホールディングス）に所有する全株式を譲渡し、同社の完全子会社となった。「ぽすれん」「ゲオ宅配レンタル」のオンラインDVD・CD・コミックレンタルサービス及び「GEO Online」と「ゲオアプリ」のアプリ・ウェブサイト運営の大きく分けて2事業を展開している。以前はDVD販売等のEコマースサービス「ぽすれんストア」、動画配信コンテンツ「ぽすれんBB」や電子書籍配信サービスの「GEO☆Books」事業も行っていた。オンラインDVDレンタル事業では会員数は10万人（2005年9月時点）。2006年5月よりCDレンタルを開始。同業他社には、カルチュア・コンビニエンス・クラブが運営する『TSUTAYA DISCAS』のほか、DMM.comが運営する『DMM.com オンラインDVDレンタル』がある。過去には「Yahoo!レンタルDVD」と「楽天レンタル」の運営を受託していた。',
 '2005年の一時期、東京のラジオ局、InterFMで、「堀江社長も使っているライブドアのぽすれん」というキャッチコピーでラジオCMを頻繁に行っていたことがあった。',
 '香川県内の農業協同組合の信用事業を統括する県域農協系金融機関であり、県内農業協同組合を会員とする。香川県は全県単一農協の香川県農業協同組合となったが、先に単一農協となった奈良県や沖縄県のケースと異なり、信連の統合は行われなかった。通称は「JA香川信連」または「JAバンク香川」。統一金融機関コードは3037。主に法人顧客を中心としており、個人取引は殆どない。県内の大型商業施設にある、他金融機関管理の共同ATMには香川信連の管轄のものがある。',
 '534年（永熙3年）、独孤信の子として生まれた。独孤信が父母妻子を捨てて長安に入ったため、独孤羅は東魏に取り残されて高氏の虜囚となった。独孤

多すぎるので減らす。

In [5]:
data = data[:1000]

これらを一つの.txtファイルに書き出しておく。後で使う。

In [6]:
textfile = 'data/jawiki_1000.txt'
with open(textfile, 'w') as f:
    for sentence in data:
        f.write(sentence + '\n')


---

## 前処理

テキストをNNで扱える形に変換する。

### トークン化

文章をトークンごとに分割する。トークンとは文章を構成する最小の単位で、単語や句読点などが該当する。  
トークン化は分かち書きと似た意味であるが、言語モデルの領域で最小単位をトークンと呼ぶことが一般的なことや、単純にトークン化と呼ぶことが多いことから、トークン化と呼ぶ。

#### 英語

英語のトークン化は簡単で、スペースで区切ればいい。

In [7]:
text = 'I am a student'
tokens = text.split(' ')
tokens

['I', 'am', 'a', 'student']

#### 日本語

日本語の場合は文章がスペースで区切られていないので、別の手法が必要。  
形態素解析器を用いるのが一般的。

In [8]:
text = '今日はいい天気だ'
tagger = MeCab.Tagger('-Owakati')
tokens = tagger.parse(text).split(' ')
tokens

['今日', 'は', 'いい', '天気', 'だ', '\n']

全ての文章をトークン化しよう。

In [9]:
data_tokens = []
for sentence in data:
    tokens = tagger.parse(sentence).split()
    data_tokens.append(tokens)

data_tokens[0][:10] # example

['「', '教科', '書', 'に', 'は', '決して', '載ら', 'ない', '」', '日本']

### ID化

NNでは文字列を扱えないので、各トークンにIDとして整数値を割り当てる。

torchtextの`vocab`モジュールを使用する。  
[torchtext.vocab — Torchtext 0.15.0 documentation](https://pytorch.org/text/stable/vocab.html#)

In [10]:
from torchtext.vocab import vocab, build_vocab_from_iterator

単語とその出現頻度を表した辞書を入力し、`Vocab`オブジェクトを作成する。`min_freq`で指定した出現頻度以下の単語は無視される。

In [11]:
v = vocab({'私': 4, 'りんご': 1, '食べる': 2, '好き': 3}, min_freq=1)
v(['私', 'りんご', '好き'])

[0, 1, 3]

`build_vocab_from_iterator()`でトークンのリスト（のリスト）から`vocab`オブジェクトを作成できる。

In [12]:
v = build_vocab_from_iterator([
    ['私', 'は', 'りんご', 'が', '好き'],
    ['私', 'は', 'バナナ', 'を', '食べる'],
    ['今日', 'は', '晴れ', 'です']
])
v.get_itos() # vocabrary

['は', '私', 'が', 'です', 'りんご', 'を', 'バナナ', '今日', '好き', '晴れ', '食べる']

これを使ってトークン列をID列に変換する。

In [13]:
data_ids = []
v = build_vocab_from_iterator(data_tokens)
for tokens in data_tokens:
    data_ids.append(v(tokens))

n_vocab = len(v)
print('num of vocabrary:', n_vocab)
data_ids[0][:10] # example

num of vocabrary: 22427


[18, 7079, 583, 2, 6, 10352, 21655, 44, 19, 66]


---

## 学習データ

NNの学習を行うため、入力と出力のペアを作成する。  
今回はある単語から次に続く単語を予測するモデルを作成するので、ある単語IDとその次の単語IDがペアとなったデータを作成する。

In [14]:
class TextDataset(Dataset):
    def __init__(self, data_ids):
        x, y = [], []
        for ids in data_ids:
            for in_id, out_id in zip(ids[:-1], ids[1:]):
                x.append(in_id)
                y.append(out_id)
        self.x = x
        self.y = y
        self.n_data = len(x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

    def __len__(self):
        return self.n_data

batch_size = 64
dataset = TextDataset(data_ids)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

n_data = len(dataset)
print('num of data:', n_data)

# examples
x, y = next(iter(dataloader))
print(x)
print(y)

num of data: 243396
tensor([    3,    14,    48,  3214,   149,  1057,   362, 20574,    31,    17,
            2,    40,   763,  3547,  9894,  5652,   104,    58,   743,   781,
            8,     1,   315,    10,  1726,   489,    93, 20353,   212,    12,
         2794,     3,    22,   324,   131,    27, 11079,   143,     9,   794,
           13,     4,     3,  2433, 13563,     1,    20,   843,    35,  1053,
           21,    36,    20,     2,    10,     8,   538,    91,  2480,     1,
            1,  6758,     8,    19])
tensor([   36, 15623,     2,   852,  5331,  3880,    72,     2,     2,  2358,
           55,     3,     4,   130,   112,     0,    11,     1,     3,   190,
         1042, 17008,   307,    23,     2,    72,     4,     2,   282,     1,
         2525,    98,   394,    12,    25,  1163,     5,   136,     1,    21,
            4,    18,  1403,  2622,   369,    88,  1716,    56,   869,     6,
          604,   185,     1,    32,    45,  1396,     5,     2,    27,   113,
       


---

## モデル構築

ある単語IDを入力に取り、次の単語IDを出力するモデルを作成する。

単語IDはカテゴリ変数なので、入力時はone-hotベクトルに変換する。出力は語彙数分の次元を持つベクトルとする。つまりこのタスクは単語の分類問題となる。

In [15]:
class LanguageModel(nn.Module):
    def __init__(self, n_vocab, hidden_size):
        super().__init__()
        self.n_vocab = n_vocab
        self.net = nn.Sequential(
            nn.Linear(n_vocab, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_vocab)
        )

    def forward(self, x):
        x = F.one_hot(x, self.n_vocab).to(torch.float32)
        y = self.net(x)
        return y


---

## 学習

損失関数に交差エントロピーを設定し、通常の分類モデルと同じように学習する。

In [16]:
criterion = nn.CrossEntropyLoss()
def train(model, optimizer, n_epochs, prog_unit=1):
    model.train()
    prog.start(n_iter=len(dataloader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(1, n_epochs + 1):
        for x, t in dataloader:
            optimizer.zero_grad()
            x = x.to(device)
            t = t.to(device)
            y = model(x)
            loss = criterion(y, t)
            loss.backward()
            optimizer.step()
            prog.update(loss.item())

In [17]:
hidden_size = 512
model = LanguageModel(n_vocab, hidden_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [18]:
train(model, optimizer, 20, 2)

  1-2/20: ######################################## 100% [00:00:53.69] loss: 5.59791  
  3-4/20: ######################################## 100% [00:00:52.62] loss: 4.35564 
  5-6/20: ######################################## 100% [00:00:52.50] loss: 3.89540 
  7-8/20: ######################################## 100% [00:00:52.51] loss: 3.74547 
 9-10/20: ######################################## 100% [00:00:52.48] loss: 3.69052 
11-12/20: ######################################## 100% [00:00:52.49] loss: 3.66524 
13-14/20: ######################################## 100% [00:00:52.42] loss: 3.65046 
15-16/20: ######################################## 100% [00:00:52.44] loss: 3.64009 
17-18/20: ######################################## 100% [00:00:52.65] loss: 3.63196 
19-20/20: ######################################## 100% [00:00:52.57] loss: 3.62591 



---

## 文章生成

学習したモデルを用いて文章を生成する。

学習させたモデルは、ある単語から次の単語を予測するモデルである。厳密には、ある単語IDを入力に取り、次の単語IDを出力するモデルである。  
さらに厳密に言うと、出力は単語IDではなく確率分布である。この確率分布から次の単語IDをサンプリングすることで、次の単語を生成する。

実際に文章を生成させてみる。ある単語からの次の単語の予測を繰り返すことで文章を生成する。初めの単語だけはこちらで指定する。  
以下の条件を満たすまで単語の生成を続ける。
- 単語数が指定した限界に達する
- 句点が出力される

In [19]:
end_token_id = v.get_stoi()['。'] # 読点のID

def generate_sentence(model, start_word, max_len=30):
    model.eval()
    token_id = v.get_stoi()[start_word]
    token_ids = [token_id]

    # 終了条件を満たすまで単語を生成
    while len(token_ids) <= max_len and token_id != end_token_id:

        # 入力する単語IDをtensorに変換
        x = torch.tensor(token_id).unsqueeze(0).to(device)

        # 次の単語の確率分布を予測
        y = model(x)[0]
        y = F.softmax(y, dim=0)

        # 分布に基づいたサンプリングを行う
        token_id = random.choices(range(len(y)), weights=y)[0]
        token_ids.append(token_id)

    tokens = [v.get_itos()[i] for i in token_ids]
    sentence = ''.join(tokens)
    return sentence

In [20]:
start_words = ['今日', '地球', '科学', '人']
for start_word in start_words:
    print(generate_sentence(model, start_word))

今日、この軽巡洋艦歴はリトアニアの流動性K.5月からスタンウェイの姿勢は、誌面でマークして白石、国境
地球温暖化、陽イオン自身の魅力的に記され、ヨネダは彼は1751年にはアジア北西のキリスト教徒という説明
科学的に溝を超えた父の政策に覆うフェアリングの重要のカリフォルニア大学リーグだった要塞施設は知名度もはっきりと改めた
人が随時公表を持つことにあたったことに子孫が、カナダを優先していく。


マルコフモデル同様、直前の単語のみを予測に用いているため、不自然な文章が多く生成される。


---

## 埋め込み層

*Embedding Layer*

NN内部における、単語IDに対する「one-hotベクトル化→線形変換」を行う部分をまとめて**埋め込み層**と表す。  
そもそも、これらの処理は単語IDを受け取って対応するベクトルを出力すること=**単語のベクトル化**と同義である。自然言語処理の世界では単語のベクトル化を**単語の埋め込み**と表現する。

線形変換を行う全結合層は各単語の埋め込み表現（単語ベクトル）を保有していることになる。それらは重みから確認できる。

In [21]:
class Embedding(nn.Module):
    def __init__(self, n_vocab, embed_dim):
        super().__init__()
        self.n_vocab = n_vocab
        self.fc = nn.Linear(n_vocab, embed_dim, bias=False)

    def forward(self, x):
        x = F.one_hot(x, self.n_vocab).to(torch.float32)
        h = self.fc(x)
        return h

embedding = Embedding(n_vocab=3, embed_dim=5)

x = torch.tensor([0, 1, 2])
h = embedding(x)

print(h, '\n') # 埋め込み層からの出力
print(embedding.fc.weight.T) # 全結合層の重み

tensor([[-0.1121, -0.2213, -0.1408,  0.0932, -0.2149],
        [-0.4840, -0.5677, -0.3134, -0.2998,  0.1772],
        [-0.1979, -0.1080,  0.0571,  0.4933,  0.1174]], grad_fn=<MmBackward0>) 

tensor([[-0.1121, -0.2213, -0.1408,  0.0932, -0.2149],
        [-0.4840, -0.5677, -0.3134, -0.2998,  0.1772],
        [-0.1979, -0.1080,  0.0571,  0.4933,  0.1174]],
       grad_fn=<PermuteBackward0>)


`PyTorch`には`torch.nn.Embedding`というクラスが実装されているので、それを使うと良い。  
[Embedding — PyTorch 2.0 documentation](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)

In [22]:
class LanguageModel(nn.Module):
    def __init__(self, n_vocab, hidden_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Embedding(n_vocab, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_vocab)
        )

    def forward(self, x):
        y = self.net(x)
        return y

実際に、言語モデルの学習によって得られる単語ベクトルは良い埋め込み表現として機能することが知られている。単語の埋め込み表現を得ることを目的として言語モデルを学習させることもある。word2vecなどがこれに該当し、これについてはおまけの章で取り上げる。


---

## サブワード分割

よりよいトークン化の手法。  
データセットから頻出する文字の並びを学習し、その並びをトークンとして分割する。

形態素解析器を用いたやり方は、辞書が必要なことやトークンの種類が多くなるなどの問題がある。トークンの種類が多くなると必要なパラメータが多くなるので、減らしていきたい。  
サブワード分割はデータセットに合ったトークン化を実現する。語彙数を指定することも可能。指定した語彙数に収まるまで細かく分割する。

In [23]:
import sentencepiece as spm

### 学習

In [ ]:
tokenizer_prefix = 'models/jawiki_1000_tokenizer'
vocab_size = 8000 # 語彙数
spm.SentencePieceTrainer.Train(
    input=textfile,
    model_prefix=tokenizer_prefix,
    vocab_size=vocab_size
)

### トークン化

In [25]:
sp = spm.SentencePieceProcessor(f'{tokenizer_prefix}.model') # モデル読み込み
text = '今日はいい天気だ'
print('tokens:', sp.encode(text, out_type=str))
print('ids:', sp.encode(text, out_type=int))

tokens: ['▁', '今日', 'は', 'いい', '天', '気', 'だ']
ids: [11, 1993, 6, 2317, 681, 343, 263]


アンダーバーは空白を意味する。初めにアンダーバーが付くのは仕様。

全てのデータをトークン化しよう。

In [35]:
data[:5] # 再掲

['「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。',
 'ライブドア社員であった初代代表取締役社長の山名真由によって企業内起業の形で創業。2005年に株式会社ライブドアから分割されて設立。かつてはライブドアホールディングス（現・LDH）の子会社であったが、ノンコア事業の整理にともない、株式会社ゲオ（現：株式会社ゲオホールディングス）に所有する全株式を譲渡し、同社の完全子会社となった。「ぽすれん」「ゲオ宅配レンタル」のオンラインDVD・CD・コミックレンタルサービス及び「GEO Online」と「ゲオアプリ」のアプリ・ウェブサイト運営の大きく分けて2事業を展開している。以前はDVD販売等のEコマースサービス「ぽすれんストア」、動画配信コンテンツ「ぽすれんBB」や電子書籍配信サービスの「GEO☆Books」事業も行っていた。オンラインDVDレンタル事業では会員数は10万人（2005年9月時点）。2006年5月よりCDレンタルを開始。同業他社には、カルチュア・コンビニエンス・クラブが運営する『TSUTAYA DISCAS』のほか、DMM.comが運営する『DMM.com オンラインDVDレンタル』がある。過去には「Yahoo!レンタルDVD」と「楽天レンタル」の運営を受託していた。',
 '2005年の一時期、東京のラジオ局、InterFMで、「堀江社長も使っているライブドアのぽすれん」というキャッチコピーでラジオCMを頻繁に行っていたことがあった。',
 '香川県内の農業協同組合の信用事業を統括する県域農協系金融機関であり、県内農業協同組合を会員とする。香川県は全県単一農協の香川県農業協同組合となったが、先に単一農協となった奈良県や沖縄県のケースと異なり、信連の統合は行われなかった。通称は「JA香川信連」または「JAバンク香川」。統一金融機関コードは3037。主に法人顧客を中心としており、個人取引は殆どない。県内の大型商業施設にある、他金融機関管理の共同ATMには香川信連の管轄のものがある。',
 '534年（永熙3年）、独孤信の子として生まれた。独孤信が父母妻子を捨てて長安に入ったため、独孤羅は東魏に取り残されて高氏の虜囚となった。独孤

In [26]:
data_ids = sp.encode(data)
n_vocab = len(sp)
print('num of vocabrary:', n_vocab)
data_ids[0][:10] # example

num of vocabrary: 8000


[11, 18, 6254, 54, 1057, 58, 1685, 79, 122, 17]

ちなみに、ChatGPTがどの様にトークン化しているかをこのサイトから試せる: [OpenAI Platform](https://platform.openai.com/tokenizer)


---

## 特殊トークン

言語モデルを扱う際、データを扱いやすくするために特殊なトークンを考えることがある。  
以下に例を示す。

### *Unknown*

未知語を意味するトークン。  
推論時に学習データに含まれなかった単語に出会ったときに対応できるようになる。学習データ内での出現回数が少ない単語も未知語として扱うことがある。

### *Begin of Sentence*

文章の先頭を意味するトークン。  
先程の例では初めの単語を与える必要があったが、このトークンを作ることで、モデルに「文章の初め」を伝えられるようになる。

### *End of Sentence*

文章の終わりを意味するトークン。  
先程の例では句点が出た場合に生成を止めたが、本来句点は文の終わりであって文章の終わりではない。このトークンを作ることでモデルが文章の終わりを伝えられるようになる。

学習データの全ての文章の該当箇所（BOSであれば文章の初め、EOSであれば文章の終わり）にこれらのトークンを入れてから学習させることで、そのトークンの意味をモデルは理解する。  
またモデルが直接触れるのはトークンではなくトークンIDなので、特殊トークンの名前は何でもいい。他のトークンと重複しないように括弧を付けることが多い。

<br>

では特殊トークンを使って学習させてみよう。使う特殊トークンは上の三つ。

In [ ]:
UNK = '[UNK]' # Unknown
BOS = '[BOS]' # Begin of sentence
EOS = '[EOS]' # End of sentence

spm.SentencePieceTrainer.Train(
    input=textfile,
    model_prefix=tokenizer_prefix,
    vocab_size=vocab_size,
    unk_piece=UNK,
    bos_piece=BOS,
    eos_piece=EOS
)

なお`sentencepiece`では上記3つのトークンがID0, 1, 2にデフォルトで設定されているので、本当は指定しなくてもよい。

In [ ]:
# 何も指定せずに学習
spm.SentencePieceTrainer.Train(
    input=textfile,
    model_prefix=tokenizer_prefix,
    vocab_size=vocab_size,
)

In [29]:
# ID0, 1, 2
sp = spm.SentencePieceProcessor(f'{tokenizer_prefix}.model')
for i in range(3):
    print(f'ID{i}:', sp.id_to_piece(i))

ID0: <unk>
ID1: <s>
ID2: </s>


では、トークン列にBOSとEOSを追加しよう。

In [30]:
bos_id = sp.bos_id()
eos_id = sp.eos_id()
for ids in data_ids:
    ids[0] = bos_id # 先頭にBOSを追加, アンダーバーは上書き
    ids[-1] = eos_id

残りを学習させる。

In [31]:
dataset = TextDataset(data_ids)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
model = LanguageModel(len(v), 512).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [32]:
train(model, optimizer, 20, 2)

  1-2/20: ######################################## 100% [00:00:52.00] loss: 6.30764  
  3-4/20: ######################################## 100% [00:00:51.88] loss: 4.90478 
  5-6/20: ######################################## 100% [00:00:51.82] loss: 4.71638 
  7-8/20: ######################################## 100% [00:00:51.85] loss: 4.63881 
 9-10/20: ######################################## 100% [00:00:51.78] loss: 4.59391 
11-12/20: ######################################## 100% [00:00:51.59] loss: 4.56537 
13-14/20: ######################################## 100% [00:00:50.83] loss: 4.54433 
15-16/20: ######################################## 100% [00:00:49.23] loss: 4.52929 
17-18/20: ######################################## 100% [00:00:49.13] loss: 4.51622 
19-20/20: ######################################## 100% [00:00:49.11] loss: 4.50476 


では文章を生成する。初めのトークンはBOSとし、EOSが出力されるまで生成を続ける。

In [33]:
unk_id = sp.unk_id()

def generate_sentence(model, max_len=50):
    model.eval()
    token_id = bos_id
    token_ids = []

    for _ in range(max_len):
        x = torch.tensor(token_id).unsqueeze(0).to(device)
        y = model(x)[0]
        y[unk_id] = -torch.inf # UNKがサンプリングされる確率を0にする
        y = F.softmax(y, dim=0)
        token_id = random.choices(range(len(y)), weights=y)[0]
        if token_id == eos_id:
            break
        token_ids.append(token_id)

    sentence = sp.decode(token_ids)
    return sentence

In [34]:
for _ in range(5):
    print(generate_sentence(model))

」(Budelep スケ、新において、後に銀塩田になり、ゴールへの危機をを行い、美少女たちを中心に、思い出部と例外見孝二駆ド州行について、小学校設置に結びつ
立地盤木市方面軍司令官(後の207月3つのシングル4チーム名の実家からの中であった。校が、9月12位と司政時代へと関心のおかげでインターネット間書店 tocrolso
2016年の設立以降、王潮は唐使うコンスタンティヌス1世は外が発行が総合優勝を飾る記事などに分類されているという階の日本選に到着点者役に前作『いコンクール」、「紅海、に終わった。一方、部屋号
方面に向かう。コンスタンティヌス1面はポーズに移籍した。1961世は今も環境に対する権威の通信大学の問題などが1,0年代を施できる範囲は木・以上の論文が産社会主義共和国の辞任を要求する声明を実現と、テレビ
木学識を得たほか乗車を支払いでチーム数向けの旧エルサレム)島(現状を示す著作『RL)などより、甲斐から良く守的であった山口組執り、これに並列では、実現する
